In [ ]:
from mat4py import loadmat
import pandas as pd
import glob
import matplotlib.pyplot as plt

folder_name_adhd = ["ADHD_part1/", "ADHD_part2/"]

# Creating .mat files list 
matfiles_adhd = []
for folder in folder_name_adhd:
    for file in glob.glob(folder+"*.mat"):
        matfiles_adhd.append(file)
print((matfiles_adhd))

# Reading data from .mat file and creating list of row and appending from every file
signalData = []
for file in matfiles_adhd:
    column_name = file.split('/')[-1].split('.')[0]
    tepmdata = loadmat(file)
    data_ = tepmdata[column_name]
    for item in data_:
        signalData.append(item)

# Creating a dataframe from the list 
adhd_data_df = pd.DataFrame(signalData)
adhd_data_df['class'] = 1
print(adhd_data_df.head)

In [ ]:
###### For Control Data
folder_name_control = ["Control_part1/", "Control_part2/"]

# Creating .mat files list 
matfiles_control = []
for folder in folder_name_control:
    for file in glob.glob(folder+"*.mat"):
        matfiles_control.append(file)
print(len(matfiles_control))

# Reading data from .mat file and creating list of row and appending from every file
signalData = []
for file in matfiles_control:
    column_name = file.split('/')[-1].split('.')[0]
    tepmdata = loadmat(file)
    data_ = tepmdata[column_name]
    for item in data_:
        signalData.append(item)
        
# Creating a dataframe from the list 
control_data_df = pd.DataFrame(signalData)
control_data_df['class'] = 0
print(control_data_df.head)


In [ ]:
column_name = ['Fp1','Fp2','F3','F4','C3','C4','P3','P4','O1','O2','F7','F8','T7','T8','P7','P8','Fz','Cz','Pz', "class"]

In [ ]:
final_df = pd.concat([adhd_data_df, control_data_df], axis=0)
final_df.columns = column_name
print(final_df.head)

In [ ]:
# Region Sepration - Right brain
right_brain_df = final_df[['Fp2','F4','F8', 'C4','T7', 'P4','T8','O2']]
print(right_brain_df)

In [ ]:
# Left brain
left_brain_df = final_df[['C3','T3','Fp1', 'F3', 'F7' , 'P3', 'T5','O1']]
print(left_brain_df)

In [ ]:
# F region
F_df= final_df[['Fz', 'Fp1', 'Fp2', 'F3', 'F4','F7', 'F8']]
print(F_df)

In [ ]:
# C region
C_df= final_df[['Cz','C3','C4']]
print(C_df)

In [ ]:
# P region
P_df= final_df[['Pz','P3','P4']]
print(P_df)

In [ ]:
# T region
T_df = final_df[['T3', 'T4', 'T5', 'T6']]
print(T_df)

In [ ]:
# O region
O_df = final_df[['O1', 'O2']]
print(O_df)

In [ ]:
c_f_o_df = pd.concat([C_df, F_df, O_df], axis=1)
c_f_o_df

In [ ]:
p_t_df = pd.concat([P_df, T_df], axis=1)
p_t_df.head()

In [ ]:
f_o_p_df = pd.concat([F_df, O_df, P_df], axis=1)
f_o_p_df

In [ ]:
c_p_t_df = pd.concat([C_df, P_df, T_df], axis=1)
c_p_t_df

In [ ]:
# F_C_P_T_O

FCPTO_df = final_df


In [ ]:
FCPTO_df.shape

### Data Scale Models

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn import metrics
from matplotlib import rcParams
rcParams['figure.figsize'] = 20,10
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()
scale_data = FCPTO_df.drop(['class'], axis=1)
scale_data = scaler.fit_transform(scale_data)


In [ ]:
scale_data.shape

In [ ]:
X = scale_data
Y = FCPTO_df['class']

In [ ]:
rf_model = RandomForestClassifier(random_state=0, n_estimators=100)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
y_probs = rf_model.predict_proba(X_test)
print(precision_recall_fscore_support(y_test, y_pred))

print(f"RandomForest Accuracy {(accuracy_score(y_test, y_pred))}")

In [ ]:
ada_model = AdaBoostClassifier(random_state=0, n_estimators=100, base_estimator=DecisionTreeClassifier())
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=0)
ada_model.fit(X_train, y_train)
y_pred = ada_model.predict(X_test)
y_probs = ada_model.predict_proba(X_test)
print(precision_recall_fscore_support(y_test, y_pred))

print(f"Adaboost Accuracy {(accuracy_score(y_test, y_pred))}")

### Feature Selection Models

In [ ]:
final = final_df.drop(['class'], axis=1)
Y = final_df['class']
data_sets = [final, right_brain_df, left_brain_df, C_df, F_df, O_df, P_df, T_df, c_f_o_df, p_t_df, f_o_p_df, c_p_t_df]
data_sets_labels = ['final_df', 'right_brain_df', 'left_brain_df', 'C_df', 'F_df', 'O_df', 'P_df', 'T_df', 'c_f_o_df', 'p_t_df', 'f_o_p_df', 'c_p_t_df']

### RandomForest Classifier

In [ ]:
classification_report_ = []

rf_model = RandomForestClassifier(random_state=0, n_estimators=100)
for label, data_set in zip(data_sets_labels, data_sets):
  print(f"==== RandomForestClassifier Modeling of {label} dataset ====")
  y_pred=[]
  y_probs=[]
  X_train, X_test, y_train, y_test = train_test_split(data_set, Y, test_size=0.20, random_state=0)
  rf_model.fit(X_train, y_train)
  y_pred = rf_model.predict(X_test)
  y_probs = rf_model.predict_proba(X_test)
  cc = []
  cc = precision_recall_fscore_support(y_test, y_pred)
  # print(cc)
  acc = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {acc}")
  classification_report_.append({ 'precision_class0_'+label: cc[0][0],
                                 'precision_class1_'+label: cc[0][1],
                                 'recall_class0_'+label: cc[1][0],
                                 'recall_class1_'+label: cc[1][1], 
                                 'f1-score_class0_'+label: cc[2][0],
                                 'f1-score_class1_'+label: cc[2][1],
                                 'Accuracy_'+label: acc
                              })



In [ ]:
# Seprating Row labels and values
row_index = []
vals = []
for index in range(len(classification_report_)):
    for key in classification_report_[index]:
        row_index.append(key)
        vals.append(round(classification_report_[index][key]*100, 2))


### AdaBoost Classifier

In [ ]:
classification_report_ada = []

ada_model = AdaBoostClassifier(random_state=0, n_estimators=100, base_estimator=DecisionTreeClassifier())
for label, data_set in zip(data_sets_labels, data_sets):
  print(f"==== AdaBoostClassifier Modeling of {label} dataset ====")
  y_pred=[]
  y_probs=[]
  X_train, X_test, y_train, y_test = train_test_split(data_set, Y, test_size=0.20, random_state=0)
  ada_model.fit(X_train, y_train)
  y_pred = ada_model.predict(X_test)
  y_probs = ada_model.predict_proba(X_test)
  cc = []
  cc = precision_recall_fscore_support(y_test, y_pred)
  # print(cc)
  acc = accuracy_score(y_test, y_pred)
  print(f"Accuracy: {acc}")
  classification_report_ada.append({ 'precision_class0_'+label: cc[0][0],
                                 'precision_class1_'+label: cc[0][1],
                                 'recall_class0_'+label: cc[1][0],
                                 'recall_class1_'+label: cc[1][1], 
                                 'f1-score_class0_'+label: cc[2][0],
                                 'f1-score_class1_'+label: cc[2][1],
                                 'Accuracy_'+label: acc
                              })

In [ ]:
vals_ada = []
for index in range(len(classification_report_ada)):
    for key in classification_report_ada[index]:
        vals_ada.append(round(classification_report_ada[index][key]*100, 2))

In [ ]:
output_df = pd.DataFrame({"RandomForest": vals, "AdaBoost": vals_ada}, index=row_index)
print(output_df, flush=False)

In [ ]:
output_df.to_csv('results.csv')